In [17]:
import cv2

cap = cv2.VideoCapture('videos/video1.mp4')  # Usa el ID de tu cámara virtual o stream
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Coordenadas que quieres probar
    x1, y1, x2, y2 = 100, 775, 350, 1020

    # Dibuja un rectángulo sobre la zona que estás extrayendo
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Extrae la región de interés
    roi = frame[y1:y2, x1:x2]

    # Muestra el frame completo con el recuadro
    cv2.imshow("Frame con ROI", frame)

    # Muestra la región de interés por separado
    cv2.imshow("ROI", roi)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [33]:
import cv2

# Abrir la captura de video (ajusta el ID si no es 1)
path = 'videos/video3.mp4'
cap = cv2.VideoCapture(path)

# Coordenadas de la región donde aparece "1st"
# Ajustalas según tu resolución/captura
x1, y1, x2, y2 = 80, 750, 350, 1050


# Esperar que el usuario confirme y guarde con tecla 's'
print("Presiona 's' para guardar el template, 'q' para salir sin guardar.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    roi = frame[y1:y2, x1:x2]
    cv2.imshow("Región recortada", roi)
    cv2.imshow("Frame completo", frame)
    if cv2.waitKey(1) & 0xFF == ord('a'):
        cv2.imwrite("template_11.png", roi)
        print("✅ Template guardado como 'template_1st.png'")
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Presiona 's' para guardar el template, 'q' para salir sin guardar.
✅ Template guardado como 'template_1st.png'


In [6]:
import cv2
import numpy as np
import time

x1, y1, x2, y2 = 80, 750, 350, 1050

# Capturar video
path = 'videos/video1.mp4'
cap = cv2.VideoCapture(path)

# Definir tus posiciones
positions = [
    {"label": "1st", "template": cv2.imread("template_1.png", 0)},
    {"label": "2nd", "template": cv2.imread("template_2.png", 0)},
    {"label": "3rd", "template": cv2.imread("template_3.png", 0)},
    {"label": "4th", "template": cv2.imread("template_4.png", 0)},
    {"label": "5th", "template": cv2.imread("template_5.png", 0)},
    {"label": "6th", "template": cv2.imread("template_6.png", 0)},
    {"label": "7th", "template": cv2.imread("template_7.png", 0)},
    {"label": "8th", "template": cv2.imread("template_8.png", 0)},
    {"label": "9th", "template": cv2.imread("template_9.png", 0)},
    {"label": "10th", "template": cv2.imread("template_10.png", 0)},
    {"label": "11th", "template": cv2.imread("template_11.png", 0)},
    {"label": "12th", "template": cv2.imread("template_12.png", 0)},
]

# Inicializar cronos
for pos in positions:
    pos["active"] = False
    pos["start_time"] = 0
    pos["total_time"] = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    roi = gray[y1:y2, x1:x2]
    now = time.time()

    best_match = None
    best_score = 0

    # Buscar la mejor coincidencia
    for pos in positions:
        res = cv2.matchTemplate(roi, pos["template"], cv2.TM_CCOEFF_NORMED)
        score = np.max(res)
        if score > best_score and score >= 0.6:
            best_match = pos
            best_score = score

    # Actualizar cronómetros
    for pos in positions:
        if pos is best_match:
            if not pos["active"]:
                pos["active"] = True
                pos["start_time"] = now
        else:
            if pos["active"]:
                pos["active"] = False
                pos["total_time"] += now - pos["start_time"]

    # ======= Ventana de estadísticas =======
    stats_img = np.ones((800, 400, 3), dtype=np.uint8) * 20  # fondo negro/gris oscuro
    for i, pos in enumerate(positions):
        current = pos["total_time"]
        if pos["active"]:
            current += now - pos["start_time"]
        color = (0, 255, 0) if pos["active"] else (150, 150, 150)
        cv2.putText(stats_img, f"{pos['label']}: {current:.2f}s", (20, 50 + i * 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
    cv2.imshow("Mario Kart Tracking", frame)
    cv2.imshow("Estadísticas", stats_img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Finalizar cualquier estado activo
now = time.time()
for pos in positions:
    if pos["active"]:
        pos["total_time"] += now - pos["start_time"]

cap.release()
cv2.destroyAllWindows()

# Mostrar resultados
print("\n=== Resultados finales ===")
for pos in positions:
    print(f"⏱️ {pos['label']}: {pos['total_time']:.2f} s")


=== Resultados finales ===
⏱️ 1st: 0.00 s
⏱️ 2nd: 0.00 s
⏱️ 3rd: 0.00 s
⏱️ 4th: 0.00 s
⏱️ 5th: 2.12 s
⏱️ 6th: 1.81 s
⏱️ 7th: 3.22 s
⏱️ 8th: 0.00 s
⏱️ 9th: 0.45 s
⏱️ 10th: 0.00 s
⏱️ 11th: 0.00 s
⏱️ 12th: 0.00 s


In [45]:
import cv2
import numpy as np
import time

x1, y1, x2, y2 = 80, 750, 350, 1050

# Capturar video
path = 'videos/video_3p.mp4'
cap = cv2.VideoCapture(path)

# Definir tus posiciones
positions = [
    {"label": "1st", "template": cv2.imread("template_1.png", 0)},
    {"label": "2nd", "template": cv2.imread("template_2.png", 0)},
    {"label": "3rd", "template": cv2.imread("template_3.png", 0)},
    {"label": "4th", "template": cv2.imread("template_4.png", 0)},
    {"label": "5th", "template": cv2.imread("template_5.png", 0)},
    {"label": "6th", "template": cv2.imread("template_6.png", 0)},
    {"label": "7th", "template": cv2.imread("template_7.png", 0)},
    {"label": "8th", "template": cv2.imread("template_8.png", 0)},
    {"label": "9th", "template": cv2.imread("template_9.png", 0)},
    {"label": "10th", "template": cv2.imread("template_10.png", 0)},
    {"label": "11th", "template": cv2.imread("template_11.png", 0)},
    {"label": "12th", "template": cv2.imread("template_12.png", 0)},
]

w, h = positions[0]['template'].shape[::-1]

# Inicializar cronos
for pos in positions:
    pos["active"] = False
    pos["start_time"] = 0
    pos["total_time"] = 0

cnt = 0
while True:
    cnt += 1
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    roi = gray[y1:y2, x1:x2]
    now = time.time()

    best_match = []

    if cnt > 30:
        # Buscar la mejor coincidencia
        for pos in positions:
            res = cv2.matchTemplate(gray, pos['template'], cv2.TM_CCOEFF_NORMED)
            threshold = 0.65
            loc = np.where(res >= threshold)
            print(pos)

            # Dibujar rectángulos donde haya match
            for pt in zip(*loc[::-1]):
                print(pt)
                cv2.rectangle(frame, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)
        
        cnt = 0

    

    
    cv2.imshow("Tracking de posiciones", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Finalizar cualquier estado activo
now = time.time()
for pos in positions:
    if pos["active"]:
        pos["total_time"] += now - pos["start_time"]

cap.release()
cv2.destroyAllWindows()

# Mostrar resultados
print("\n=== Resultados finales ===")
for pos in positions:
    print(f"⏱️ {pos['label']}: {pos['total_time']:.2f} s")

{'label': '1st', 'template': array([[61, 63, 66, ..., 63, 63, 63],
       [66, 70, 72, ..., 63, 63, 64],
       [71, 72, 73, ..., 64, 64, 64],
       ...,
       [80, 80, 80, ..., 92, 92, 92],
       [80, 80, 80, ..., 92, 92, 92],
       [79, 79, 79, ..., 92, 92, 92]], shape=(300, 270), dtype=uint8), 'active': False, 'start_time': 0, 'total_time': 0}
{'label': '2nd', 'template': array([[120, 120, 122, ..., 125, 125, 123],
       [122, 122, 123, ..., 125, 125, 125],
       [124, 125, 127, ..., 128, 126, 126],
       ...,
       [151, 150, 143, ..., 153, 155, 155],
       [149, 148, 142, ..., 155, 155, 156],
       [148, 146, 142, ..., 155, 155, 157]], shape=(300, 270), dtype=uint8), 'active': False, 'start_time': 0, 'total_time': 0}
{'label': '3rd', 'template': array([[103, 103, 102, ..., 110, 110, 110],
       [103, 103, 102, ..., 110, 110, 110],
       [103, 103, 102, ..., 110, 110, 110],
       ...,
       [106, 106, 107, ..., 105, 105, 103],
       [106, 106, 107, ..., 106, 106, 104

KeyboardInterrupt: 